### Concatenacion de features

En esta porcion de codigo se describe la secuencia de pasos necesario para añadir las 30 mediciones anteriores observadas para cada tupla. Los criterios o casos a considerar son los siguientes:

+ **Caso ideal:** existen mediciones observadas para 30 dias anteriores. En caso de haber mas de 1 observacion por dia el resultado final a considerar sera el promedio de estas.
+ **Caso medio:** existen mediciones observadas para algunos dias anteriores. Igual que en el caso anterior, si existe mas de una observacion diaria se devuelve el promedio de estas. Se termina rellenando hasta completar los 30 features con el promedio de los valores ya observados, es decir, si tenemos 5 observaciones, los 25 features restantes seran rellenados con el promedio de esas 5 observaciones.
+ **Caso no ideal:** solo se tiene la observacion de la tupla en estudio. En este caso se rellenan los 30 valores con la velocidad promedio de la tupla.

**Para todos los casos se toman en cuenta las mediciones realizadas el mismo dia, pero de horas anteriores al de la tupla. Estas se terminan promediando tambien.**

In [1]:
# Importamos las librerias
import os
import pandas as pd
import numpy as np

A partir de este punto se trabajara con un nuevo dataset, el cual contiene dos nuevas columnas, indicando la hora y el minuto en que se registro la medicion. El procedimiento que generó este nuevo dataset puede encontrarse [aqui](https://github.com/yieniggu/CTL-UNAB/blob/master/03-%20Data%20Preprocessing2.py) (12-31). 

In [2]:
# Leemos el dataset
cuad_data = pd.read_csv(os.getcwd() + "/cuad_data2.csv", delimiter = ";")

In [3]:
# Ordenamos por cuadrante y fecha (dia del año)
sorted_cuad_data = cuad_data.sort_values(by=["id_Cuadrante", "year_day"])

#Visualizacion
sorted_cuad_data.head()

,id_Arco,id_Cuadrante,Nombre_Comuna,Fecha,Hora,Velocidad_Promedio,N_puntos,N_vehiculos,week_day,year_day,year_week,mean_v(n-t),Hora2,Minuto
0,126000782,41,Lampa,2018-01-30 00:00:00,2019-04-19 11:30:00,52,1,1,1,30,5,52.0,11,30
1,163825396,92,Lampa,2018-07-06 00:00:00,2019-04-19 21:45:00,10,1,1,4,187,27,10.0,21,45
2,163825399,93,Lampa,2018-01-08 00:00:00,2019-04-19 14:30:00,12,1,1,0,8,2,15.5,14,30
3,86226506,93,Lampa,2018-01-08 00:00:00,2019-04-19 14:30:00,19,1,1,0,8,2,15.5,14,30
4,163825399,93,Lampa,2018-05-24 00:00:00,2019-04-19 13:45:00,48,1,1,3,144,21,48.0,13,45


### Funcion para añadir las velocidades correspondientes

La siguiente funcion permite hacer la seleccion de velocidades a concatenar para cada tupla bajo los criterios o casos descritos mas arriba. Para los 5 casos que se pueden visualizar arriba la configuracion seria la siguiente:

+ 52, ....., 52 (Caso no ideal, solo una medicion)
+ 10, ....., 10 (Caso no ideal, solo una medicion)
+ 12, ....., 12 (Caso no ideal, con una variacion, existen otros casos para el cuadrante pero no cumplen los criterios temporales)

etc.

**La funcion considera la resta entre el dia del año de la tupla y los dias del año de las demas tuplas del mismo cuadrante. Esta debe ser 0 <= tupla <= 30.**

**En caso de existir mediciones en el mismo dia en que fue registrada la medicion para una tupla, se consideran solo aquellas que fueran medidas en instantes anteriores.**

In [4]:
def add_velocities(data):
    #values = []
    values = pd.DataFrame()
    for index, row in data.iterrows():
        cuad, day, hour, minute = row.id_Cuadrante, row.year_day, row.Hora2, row.Minuto
        
        # Se obtienen las tuplas que tengan informacion del mismo cuadrante y de dias anteriores (a lo mas 30 dias)
        local_values = data.loc[(data["id_Cuadrante"] == cuad) & (day - data["year_day"] <= 30) & (day - data["year_day"] >= 0)]
        
        # Se elimina la tupla en estudio, asi como aquellas que se midieron en el mismo instante (hora y minuto)
        local_values = local_values.loc[(local_values["year_day"] != day) | ((local_values["year_day"] == day) & (local_values["Hora2"] == hour) & (local_values["Minuto"] < minute))]
        
        # Agrupamos por dia y obtenemos la media de las observaciones
        local_values = local_values.groupby("year_day")["Velocidad_Promedio"].mean().tolist()
        
        # Reordenamos para tener temporalidad bien definida
        local_values.reverse()
        
        # Caso 1: solo existe una medicion para ese cuadrante
        if(len(local_values) == 0):
            mean = row.Velocidad_Promedio # Se rellena con el valor de la unica medicion observada
        # Caso 2: No existen suficientes mediciones previas en el intervalo de 30 dias
        elif(len(local_values) < 30):
            mean = np.array(local_values).mean() # Se rellena con la media de las observaciones disponibles
            
        while(len(local_values) < 30):
            local_values.append(mean) # Rellenamos
        
        #values.append(np.array(local_values).reshape(1,-1))
        
        local_values = np.array(local_values).reshape(1, -1)
        values = values.append(pd.DataFrame(local_values)) # Transformamos a vector fila
        
    return values

In [ ]:
# Obtenemos un dataframe con la misma cantidad de filas que el dataset
values = add_velocities(sorted_cuad_data)

#Con esto podemos guardar los features en un archivo csv para su posterior uso, en caso de algun fallo en el sistema.